In [26]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [27]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(Path("Resources/lending_data.csv"))

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [28]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X=lending_data_df.drop(columns=['loan_status'])

In [29]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [30]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [31]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [32]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state = 1, stratify = y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [33]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [34]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test)
print(testing_predictions[:5])

[0 0 0 0 0]


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [35]:
# Print the balanced_accuracy score of the model
test_balanced_accuracy_score = balanced_accuracy_score(y_test, testing_predictions)
print(test_balanced_accuracy_score)

0.9442676901753825


In [36]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, testing_predictions)
print(test_matrix)

[[18679    80]
 [   67   558]]


In [37]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Since predicting both healthy and high-risk oans are important to us, the f1-score will be the best indicator as to how well the model preforms. For healthy loans, the model has an F1-score of 1.00 meaning it predicted all of the healthy loans correctly with the test data. For the high-risk data, the model did pretty well with a score of 0.88 meaning it got 88% of the high-risk loans correct so the model does pretty well predicting both healthy and high-risk loans with a low chance of predicting high-risk loans incorrectly. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [38]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
sampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

In [39]:
# Count the distinct values of the resampled labels data
X_resampled.nunique()

loan_size            178
interest_rate       4337
borrower_income      634
debt_to_income       634
num_of_accounts       17
derogatory_marks       4
total_debt           634
dtype: int64

In [40]:
y_resampled.nunique()

2

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [45]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (58152, 7)
Shape of y_train: (58152,)


In [48]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lrm = LogisticRegression(random_state=1)

#Split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state = 1, stratify = y_resampled)

# Fit the model using the resampled training data
model=lrm.fit(X_train, y_train)

# Make a prediction using the testing data
predictions = model.predict(X_test)
print(predictions[:5])

[1 1 0 1 1]


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [51]:
# Print the balanced_accuracy score of the model 
resampled_balance_accuracy = balanced_accuracy_score(y_test, predictions)
print(resampled_balance_accuracy)

0.9942073392670292


In [52]:
# Generate a confusion matrix for the model
resampled_confusion_matrix = confusion_matrix(y_test, predictions)
print(resampled_confusion_matrix)

[[13992    77]
 [   86 13984]]


In [53]:
# Print the classification report for the model
resampled_classification_report = classification_report(y_test, predictions)
print(resampled_classification_report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14069
           1       0.99      0.99      0.99     14070

    accuracy                           0.99     28139
   macro avg       0.99      0.99      0.99     28139
weighted avg       0.99      0.99      0.99     28139



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The resampled data regression model did better than the regular data regression model in that, even though it got 99% of the healthy loans correct instead of 100%, it also got 99% of the high-risk loans correct which is a huge improvment from the 89% that the regular data model got. 